# **Structured Query Language (SQL) continued...**

### **Aggregating with `GROUP BY`**

The statement `GROUP BY` tells SQL to group the data based on the value contained in a specific column 

```SQL 
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;
```

* In this case, we're grouping by each unique value in `type`
* Then, `SUM(cost)` sums up the costs of dishes in each `type` and displays the results in the output table

There are many aggregation functions you can use with `GROUPBY`: 
* `COUNT` counts the number of rows associated with each group 
* `MIN` finds the minimum value of each group 
* `MAX` finds the maximum value of each group 
* `SUM` sums across all records in each group 
* `AVG` finds the average value of each group 

We can easily computer multiple aggregations all at once: 

```SQL 
SELECT type, SUM(cost), MIN(cost), MAX(name)
FROM Dish
GROUP BY type;
```
One thing about `COUNT`
* Using `COUNT(*)` will compute the total number of rows in each group, including rows with null values

With `GROUP BY` in hand, let's update our SQL order of operations 

```SQL 
SELECT <column expression list>
FROM <table>
[WHERE <predicate>]
[GROUP BY <column list>]
[ORDER BY <column list>]
[LIMIT <number of rows>]
[OFFSET <number of rows>];
```
* We can also use the `AS` keyword to rename columns during the selection process, and that column expressions may include aggregation functions


#### **Filtering Groups**

* If we only want groups that meet certain conditions, we can use the `HAVING` keyword
* `HAVING` filters groups by applying some condition across all rows in each group 
* We interpret it as a way to keep only groups `HAVING` some condition 

The difference between `WHERE` and `HAVING`
* `WHERE` is used to filter rows 
* `HAVING` is used to filter *groups*
* `WHERE` precedes `HAVING` in a SQL query

```SQL 
SELECT type, COUNT(*)
FROM Dish
WHERE cost > 4
GROUP BY type
HAVING MAX(cost) <  10;
```

Let's again update our toolbox of possible SQL queries: 
```SQL
SELECT <column expression list>
FROM <table>
[WHERE <predicate>]
[GROUP BY <column list>]
[ORDER BY <column list>]
[LIMIT <number of rows>]
[OFFSET <number of rows>];
```


#### **Matching Text using `LIKE`**

`LIKE` operator is used to look for strings that are *like* a given string pattern 

```SQL 
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE 'Star Wars: Episode I - The Phantom Menace'
```
<br>

What if we wanted *all* Star Wars movies? `%` is the wildcard operator, meaning it "looks for any character, any number of times"

Take a look at this example: 

```SQL
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE '%Star Wars%'
LIMIT 10;
```

<br>

We *could* also use RegEx! In DATA 100 at the time of Spring 2024, we use DuckDB for SQL queries in Jupyter notebook
* We have to use `SIMILAR TO` rather than `LIKE`

```SQL 
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle SIMILAR TO '.*Star Wars*.'
LIMIT 10;
```

#### **`CAST`ing Data Types**

The `CAST` keyword is used to generate a new output column
* Each entry in this output column is the result of converting the data in an existing column to a new data type 

```SQL
SELECT primaryTitle, CAST(runtimeMinutes AS INT)
FROM Title;
```

SQL automatically names a new column according the command used to `SELECT` it, so we can rename the `CAST`ed column using the `AS` keyword 

<br>

```SQL
SELECT primaryTitle AS title, CAST(runtimeMinutes AS INT) AS minutes, CAST(startYear AS INT) AS year
FROM Title
LIMIT 5;
```

#### **`CASE`ing Data Types**

The `CASE` clause is used for conditional operations 
* It creates a new column that we can then `SELECT` to appear in the output

The syntax is as follows: 

```SQL
CASE WHEN <condition> THEN <value>
     WHEN <other condition> THEN <other value>
     ...
     ELSE <yet another value>
     END
```

* `END` is used to specify the end of the conditional statement 
* After `END` is called, SQL will continue evaluating the rest of the query as usual 

Here's an example: 

``` SQL

/* If a movie was filmed before 1950, it is "old"
Otherwise, if a movie was filmed before 2000, it is "mid-aged"
Else, a movie is "new" */

SELECT titleType, startYear,
CASE WHEN startYear < 1950 THEN 'old'
     WHEN startYear < 2000 THEN 'mid-aged'
     ELSE 'new'
     END AS movie_age
FROM Title;

```

### **`JOIN`ing Tables**

To join data across multiple tables, we can use the `JOIN` keyword

We will illustrate this by using the `cats` dataset, which consitis of the tables `s` and `t`

<img src="https://ds100.org/course-notes/sql_II/images/cats.png" alt="Image Alt Text" width="700" height="300">

To perform a join, we amend the `FROM` clause
* "`SELECT` my data `FROM` tables that have been `JOIN`ed together`

```SQL 

SELECT <column list>
FROM table_1 
    JOIN table_2 
    ON key_1 = key_2;

```

We also need to specify what column from each table we should use to determine matching entries
* By defining these keys, we provide SQL with the information it needs to pair rows of data together 

There are five main types of `JOIN`s we will use in DATA 100: 

##### **Inner Join**
* This is the type of join which `pandas` uses 
* We combine every row in our first table with its matching entry in the second table
* If a row from either table does  not have a match in the other table, it is ommitted from the output 

<img src="https://ds100.org/course-notes/sql_II/images/inner.png" alt="Image Alt Text" width="700" height="150">

##### **Cross Join**

* *All* possible combinations of rows appear in the output table, regardless of whether or not rows share a matching key 
* There is no need to specify what keys to consider in an `ON` statement 

<img src="https://ds100.org/course-notes/sql_II/images/cross.png" alt="Image Alt Text" width="700" height="400">


##### **Left Outer Join**
* *All* rows in the left table are kept in the output table 
* If a row in the right table shares a match with the left table, this row will be kept; otherwise, the rows in the right table are ommitted from the output 
* We fill in any missing values with `NULL`

<img src="https://ds100.org/course-notes/sql_II/images/left.png" alt="Image Alt Text" width="700" height="150">

##### **Right Outer Join**
* Keeps all rows in the right table 
* Rows in the left table are onl kept if they share a match in the right table 
* We fill in any missing values with `NULL`

<img src="https://ds100.org/course-notes/sql_II/images/right.png" alt="Image Alt Text" width="700" height="150">

##### **Full Outer Join**
* All rows that have a match between two tables are joined together.
* If a rows has no match in the second table, then the values of the columns for the second table are filled with `NULL`
* So, a full outer join performs an inner join *while still keeping rows* that have no match in the other table 


<img src="https://ds100.org/course-notes/sql_II/images/full.png" alt="Image Alt Text" width="700" height="150">

#### **Aliasing in Joins**

When joining tbales, we often create aliases for table names 
* We do this as it's typically easier to refer to aliases (especially with long table names)

```SQL
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON T.tconst = R.tconst;
```

### **Common Table Expressions**

For sophisticated data problems, queriescan become very complex 
* Common table expressions (CTEs) allow us to break down these complex queries into more manageable parts 
* To do so, we create temporary tables corresponding to different aspects of the problem and then reference them in the final query 

```SQL 

WITH 
table_name1 AS ( 
    SELECT ...
),
table_name2 AS ( 
    SELECT ...
)
SELECT ... 
FROM 
table_name1, 
table_name2, ...

```